# GARLIC demo

General-purpose Adaptive Richardson-Lucy Image Characterisation

# 1. General-purpose

## Import libraries and scripts

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator

import ipywidgets as widgets
from IPython.display import display

import numpy as np
from time import time
from scipy import ndimage, special

In [ ]:
import importlib
import scripts
importlib.reload(scripts)

Plotting:

In [ ]:
def new_figure(fig_name, figsize=(10, 5), nrows=1, ncols=1, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}):
    plt.close(fig_name)
    fig = plt.figure(fig_name, figsize=figsize)
    axes = fig.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                        sharex=sharex, sharey=sharey,
                        gridspec_kw=gridspec_kw
                       )
    fig.set_tight_layout(True)
    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    fig.suptitle(fig_name)
    
    return fig, axes

In [ ]:
default_cmap = plt.get_cmap("gist_earth").copy()
default_cmap.set_bad('gray')


def colour_map(ax, cblabel, data, cmap=default_cmap, norm=None, xlabel=None, x=None, ylabel=None, y=None):
    
    sigmas = np.linspace(-3, 3, 7)
    percentiles = 50 + 50 * special.erf(sigmas / np.sqrt(2))
    ticks = np.nanpercentile(data, percentiles)
    if norm is None:
        if ticks[-1] > 0:
            linthresh = np.median(data[data > 0])
            norm = colors.SymLogNorm(vmin=ticks[0], vmax=ticks[-1], linthresh=linthresh)
        else:
            norm = colors.Normalize(vmin=ticks[0], vmax=ticks[-1])

    if y is None:
        y = np.arange(data.shape[0])
    if x is None:
        x = np.arange(data.shape[1])

    im = ax.imshow(data,
                   extent=(x[0]-(x[1]-x[0])/2, x[-1]+(x[-1]-x[-2])/2, y[0]-(y[1]-y[0])/2, y[-1]+(y[-1]-y[-2])/2),
                   interpolation='nearest', origin='lower',
                   cmap=cmap,
                   norm=norm,
                  )
    #ax.set_aspect('auto')
    if xlabel is not None:
        ax.set_xlabel(xlabel)
    if ylabel is not None:
        ax.set_ylabel(ylabel)

    cb = fig.colorbar(im, ax=ax, orientation='vertical', shrink=.9)
    cb.ax.set_ylabel(cblabel)
    if ticks is not None:
        cb.ax.set_yticks(ticks=ticks, labels=[f'{value:.3g} ({percent:.1f}%)' for value, percent in zip(ticks, percentiles)])
    cb.ax.tick_params(labelsize='small')
    
    return im, cb, norm


## Read data

In [ ]:
importlib.reload(scripts.read_data)
object_name, data, true_signal = scripts.read_data.run(11, (0, 0, 1))
data_offset = np.nanmin(data)

## Parameters

In [ ]:
residual_accuracy = .01
max_iter = 100
kernel_truncation = 6

In [ ]:
peak_threshold = 1
accretion_threshold = .5

# 2. Adaptive Richardson Lucy

Find noise, source, and background scales

In [ ]:
importlib.reload(scripts.diffuse_emission)
noise_scale = scripts.diffuse_emission.find_scale(data)
source_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, noise_scale, truncate=kernel_truncation))
diffuse_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, source_scale, truncate=kernel_truncation))
print(f'Scales: noise = {noise_scale:.2f}, sources = {source_scale:.2f}, diffuse emission = {diffuse_scale:.2f}')

In [ ]:
baseline = np.nanmin(data)
print(f'baseline={baseline:4g}')

In [ ]:
importlib.reload(scripts.multiscale_RL)
smoothing_radii = np.array([noise_scale, source_scale, diffuse_scale]) / np.sqrt(8*np.log(2)) # FWHM -> Gaussian sigma
#smoothing_radii = np.array([noise_scale, source_scale])
n_radii = smoothing_radii.size

mRL = scripts.multiscale_RL.run(data - baseline, smoothing_radii)
RL = np.sum(mRL, axis=0)
m_model = np.empty_like(mRL)
for i, radius in enumerate(smoothing_radii):
    m_model[i] = ndimage.gaussian_filter(mRL[i], radius, truncate=kernel_truncation)
model = baseline + np.sum(m_model, axis=0)

In [ ]:
np.nanmean(data), np.nanmean(baseline+3*m_model, axis=1)

In [ ]:
fig, axes = new_figure('RL_reconstruction')

ax = axes[0, 0]
ax.set_ylabel('intensity')

ax.plot(data, 'k-', alpha=.5, label='data')
if true_signal is not None:
    ax.plot(true_signal, 'k-', label='true signal')

lbl = ['compact', 'extended', 'diffuse']
style = ['y-', 'g-', 'r-']
#for i in range(smoothing_radii.size):
    #ax.plot(baseline + 3*mRL[i], style[i], alpha=.25, label=f'{lbl[i]} ($\\sigma={smoothing_radii[i]:.2f}$ pix)')
    #ax.plot(baseline + smoothing_radii.size*m_model[i], style[i], alpha=.25, label=f'{lbl[i]} ($\\sigma={smoothing_radii[i]:.2f}$ pix)')
ax.plot(baseline + 3*mRL[-1], 'y-')
ax.plot(baseline + 3*m_model[-1], 'r-')
ax.plot(baseline + smoothing_radii.size*ndimage.gaussian_filter(np.fmin(m_model[-1], mRL[-1]), smoothing_radii[-1]), 'k-')

ax.plot(model, 'b-', alpha=.5, label='model')
ax.legend()

#ax.set_yscale('log')
plt.show()

# 3. Image characterisation

## Noise

In [ ]:
residual = data - model
noise = np.sqrt(ndimage.gaussian_filter(residual**2, smoothing_radii[0], truncate=kernel_truncation)) #- ndimage.gaussian_filter(residual, diffuse_scale))
mean = np.nanmean(noise)
noise = np.where(np.isfinite(noise), noise, mean)
print(f'noise: {mean:.3g} +- {np.std(noise):.3g} [{np.min(noise):.3g} - {np.max(noise):.3g}]')

## Background

In [ ]:
pixel_type = np.argmax(m_model, axis=0)

In [ ]:
background = np.fmin(m_model[-1], mRL[-1])
background = ndimage.gaussian_filter(background, smoothing_radii[-1])
background = np.nanmedian([m_model[-1], mRL[-1], background], axis=0)
background = ndimage.gaussian_filter(background, smoothing_radii[-1])
background = baseline + smoothing_radii.size*background

In [ ]:
signal = model - background
SN = signal / noise
median_SN = np.nanmedian(SN)

In [ ]:
fig, axes = new_figure('noise_and_background', figsize=(10, 6))

ax = axes[0, 0]
ax.set_ylabel('intensity')

ax.plot(data, 'k-', alpha=.25, label='data')
if true_signal is not None:
    ax.plot(true_signal, 'k-', label='true_signal')

ax.plot(model, 'b-', alpha=.5, label='model')
#ax.fill_between(np.arange(data.size), model - noise, model + noise, color='b', alpha=.1, label=f'noise: {mean:.3g}$\\pm${np.std(noise):.3g} [{np.min(noise):.3g} - {np.max(noise):.3g}]')

ax.plot(background, 'r--', alpha=.5, label='background')
ax.fill_between(np.arange(data.size), background - noise, background + noise,
                color='r', alpha=.1, label=f'noise: {mean:.3g}$\\pm${np.std(noise):.3g}')  # [{np.min(noise):.3g} - {np.max(noise):.3g}]')

#ax.plot(background + peak_threshold*noise, 'k--', label=f'peak_threshold={peak_threshold:.2f}')
#ax.plot(background + accretion_threshold*noise, 'k:', label=f'accretion={accretion_threshold:.2f}')

ax.legend()

'''
ax = axes[1, 0]
ax.set_ylabel('S/N')

#lbl = ['compact', 'extended']
#style = ['y-', 'g-', 'r-']
#for i in range(smoothing_radii.size-1):
#    ax.plot((m_model[i] - m_model[i+1]) / noise, style[i], alpha=.5, label=lbl[i])

ax.plot(signal/noise, 'b-', alpha=.2, label='diffuse-dominated')
ax.plot(np.where(pixel_type == 2, np.nan, signal/noise), 'b-', label='source-dominated')

#ax.plot(diffuse_fraction, 'r-', alpha=.5, label='bg2')
#ax.plot(source_fraction, 'b-', alpha=.5, label='src2')

ax.plot(pixel_type, 'g-')

ax.axhline(peak_threshold, c='k', ls='--', label=f'peak_threshold={peak_threshold:.2f}')
ax.axhline(0, c='k', ls=':')
ax.legend()
'''

ax.set_xlabel('wavelength / channel')
#ax.set_yscale('log')
plt.show()

In [ ]:
pixel_type

In [ ]:
raise -1

# --- TESTS

## Automatic thresholds

In [ ]:
sorted_by_SN = np.argsort(SN.flat)
sorted_signal_fraction = np.cumsum(signal.flat[sorted_by_SN])
sorted_signal_fraction /= sorted_signal_fraction[-1]
index0 = np.searchsorted(SN.flat[sorted_by_SN], 0)
accretion_threshold = np.interp(0, sorted_signal_fraction[index0:], SN.flat[sorted_by_SN][index0:])

In [ ]:
peak_threshold = max(accretion_threshold, median_SN) + np.sqrt(np.mean((SN[SN < accretion_threshold] - accretion_threshold)**2))

In [ ]:
fig, axes = new_figure('signal-to-noise_thresholds', nrows=3, figsize=(8, 6))


ax = axes[0, 0]
ax.set_ylabel('number of measurements')
#ax.set_ylabel('probability density')

ax.hist(SN, bins=np.linspace(SN[sorted_by_SN[0]], 2*peak_threshold - SN[sorted_by_SN[0]], 3*int(1 + np.sqrt(index0))))


ax = axes[1, 0]
ax.set_ylabel('cumulative flux fraction')

ax.plot(SN.flat[sorted_by_SN], sorted_signal_fraction, 'k-')
ax.axvline(accretion_threshold, c='k', ls=':', label=f'accretion_threshold = {accretion_threshold:.2f} $\\sigma$')
ax.axvline(peak_threshold, c='k', ls='--', label=f'peak_threshold = {peak_threshold:.2f} $\\sigma$')
ax.axvline(median_SN, c='k', ls='-', label=f'median = {median_SN:.2f} $\\sigma$')

ax.legend()


ax = axes[2, 0]
ax.plot(SN.flat[sorted_by_SN], SN_var_below, 'k-')
#ax.set_ylim(.03, 300)
ax.set_yscale('log')
#ax.set_ylim(-.1, 3.1)


ax.set_xlabel('signal / noise')
ax.set_xlim(SN[sorted_by_SN[0]], 3*peak_threshold - SN[sorted_by_SN[0]])
plt.show()

## refine / renormalise background

Test 1

In [ ]:
SN_below = np.sqrt(np.nanmean(SN[signal < 0]**2))
print(SN_below)
while SN_below < 1:
    background += .1 * ndimage.gaussian_filter(noise, smoothing_radii[-1])
    signal = model - background
    SN = signal / noise
    SN_below = np.sqrt(np.nanmean(SN[signal < 0]**2))
    print(SN_below)
while SN_below > 1:
    background -= .1 * ndimage.gaussian_filter(noise, smoothing_radii[-1])
    signal = model - background
    SN = signal / noise
    SN_below = np.sqrt(np.nanmean(SN[signal < 0]**2))
    print(SN_below)
    

Test 2

In [ ]:
n = 1 + np.arange(sorted_by_SN.size)
#SN_var_below = np.cumsum(SN.flat[sorted_by_SN]**2)/n + SN.flat[sorted_by_SN]**2 - 2*SN.flat[sorted_by_SN]*(np.cumsum(SN.flat[sorted_by_SN])/n)
SN_var_below = np.cumsum(signal.flat[sorted_by_SN]**2)/n + signal.flat[sorted_by_SN]**2 - 2*signal.flat[sorted_by_SN]*(np.cumsum(signal.flat[sorted_by_SN])/n)

# --- OLD

## Signal

In [ ]:
mSN = (baseline + n_radii*m_model - model[np.newaxis, :]) / noise[np.newaxis, :]
mp_signal = 1 - np.exp(-.5*mSN**2)
p_signal = ndimage.gaussian_filter(np.nanmean(mp_signal, axis=0), smoothing_radii[0], truncate=kernel_truncation)

In [ ]:
fig, axes = new_figure('signal-probability', nrows=2)

ax = axes[0, 0]
ax.set_ylabel('intensity')

lbl = ['compact', 'diffuse', 'background']
style = ['y-', 'g-', 'r-']
for i in range(smoothing_radii.size):
    ax.plot(mp_signal[i], style[i], alpha=.25, label=lbl[i])

ax.plot(p_signal, 'k-', label='mean')

ax.legend()


ax = axes[1, 0]
ax.set_ylabel('intensity')

ax.plot(data, 'k-', alpha=.5, label='observed')
if true_signal is not None:
    ax.plot(true_signal, 'k-', label='true')

lbl = ['compact', 'diffuse', 'background']
style = ['y-', 'g-', 'r-']
for i in range(smoothing_radii.size):
    ax.plot(baseline + smoothing_radii.size*m_model[i], style[i], alpha=.5, label=lbl[i])

ax.plot(model, 'b-', label='reconstruction')
ax.fill_between(np.arange(data.size), model - noise, model + noise, color='b', alpha=.1)

ax.legend()
ax.set_yscale('log')

plt.show()

In [ ]:
signal

In [ ]:
SN_map = residual / noise
SN_min = np.min(SN_map)
SN_max = max(-2*SN_min, min(np.max(SN_map), 9.5))
hist, bins = np.histogram((SN_map).ravel(), bins=np.arange(SN_min, SN_max, .1), density=True)
x_bins = (bins[1:] + bins[:-1]) / 2
index_max = np.argmax(hist)
SN_mode = x_bins[index_max]
#x_HWHM = np.interp(hist[index_max]/2, hist[:index_max], x_bins[:index_max])
#y_HWHM = 2*SN_mode - x_HWHM
#SN_sigma = (SN_mode - x_HWHM) / np.sqrt(2*np.log(2))
SN_threshold = 2*SN_mode - SN_min

In [ ]:
fig, axes = new_figure('residual')#, nrows=2)


ax = axes[0, 0]
ax.plot(x_bins, hist, 'k-+')
ax.axvline(SN_mode, c='k', ls='--', label=f'mode={SN_mode:.2f}')
#ax.axvline(x_HWHM, c='k', ls=':')
#ax.axvline(y_HWHM, c='k', ls=':')
ax.axvline(SN_threshold, c='k', ls='-.', label=f'threshold={SN_threshold:.2f}')
ax.axhline(hist[index_max]/2, c='k', ls=':')

#ax.plot(x_bins, np.exp(-.5 * ((x_bins-SN_mode)/SN_sigma)**2) / np.sqrt(2*np.pi) / SN_sigma, 'k:')
#ax.plot(x_bins, np.exp(-.5 * ((x_bins-SN_mode)/SN_sigma)**2) * hist[index_max], 'k:')
#ax.set_yscale('log')
ax.legend()

'''
ax = axes[1, 0]
ax.scatter(SN_map, background_estimate, s=1, alpha=1)
ax.set_xlim(SN_min, SN_max)
'''

plt.show()

In [ ]:
fig, axes = new_figure('signal-to-noise')#, nrows=2)


ax = axes[0, 0]
ax.plot(x_bins, hist, 'k-+')
ax.axvline(SN_mode, c='k', ls='--', label=f'mode={SN_mode:.2f}')
ax.axvline(SN_threshold, c='k', ls='-.', label=f'threshold={SN_threshold:.2f}')
ax.axhline(hist[index_max]/2, c='k', ls=':')

p16, p50 = np.nanpercentile(SN_map, [16, 50])
ax.axvline(p16, c='k', ls=':')
ax.axvline(p50, c='c', ls=':', label=f'median={p50:.2f}')
ax.axvline(2*p50-SN_min, c='k', ls=':', label=f'threshold={2*p50-SN_min:.2f}')
mu0 = np.nanmean(SN_map)
#mu0 = np.nanmean(SN_map[SN_map < mu0])
std0 = np.sqrt(np.nanmean(mu0-SN_map[SN_map < mu0])**2)
mu0_threshold = 2*mu0-SN_min
ax.axvline(mu0, c='r', ls='--', label=f'mean={mu0:.2f} +- {std0:.2f}')
ax.axvline(mu0_threshold, c='r', ls='-.', label=f'mu0_threshold={mu0_threshold:.2f}')

ax.legend()

'''
ax = axes[1, 0]
ax.scatter(residual/noise, background_estimate, s=1, alpha=.1)
ax.set_xlim(SN_min, SN_max)
'''

plt.show()

In [ ]:
noise

# 3. Source finding

## Hierarchical Overdensity Tree

In [ ]:
importlib.reload(scripts.sort_data)
#argsorted_data, n_valid = scripts.sort_data.run(RL.ravel())
argsorted_data, n_valid = scripts.sort_data.run(mRL.ravel())

In [ ]:
importlib.reload(scripts.HOT)
sorted_strides = np.hstack([np.sort(mRL.strides)//mRL.itemsize, mRL.size]) # DIRTY HACK when testig particles at the boundary
t0 = time()
HOT_labels, HOT_catalog = scripts.HOT.run(mRL, argsorted_data, sorted_strides)
#n_sources = np.unique(HOT_labels).size
n_sources = np.max(HOT_labels)
print(f'     {time()-t0:.3g} seconds')

In [ ]:
HOT_parent = HOT_catalog[0]
'''
HOT_area = HOT_catalog[1]
HOT_test_stat = HOT_catalog[2]
HOT_bg = HOT_catalog[3]
#max_test_stat = catalog[3]
'''
,

## Individual sources

In [ ]:
def get_source_model(mRL, HOT_labels, lbl):
    RL = np.zeros_like(data)
    indices = np.where(HOT_labels[0] == lbl)
    RL[indices] = mRL[0][indices]
    source_model = ndimage.gaussian_filter(RL, smoothing_radii[0])
    
    RL = np.zeros_like(data)
    indices = np.where(HOT_labels[1] == lbl)
    RL[indices] = mRL[1][indices]
    source_model += ndimage.gaussian_filter(RL, smoothing_radii[1])
    
    return source_model

In [ ]:
residual = (compact_emission + diffuse_emission - diffuse_estimate)
def compute_moment(m):
    RL_moment = mRL.copy()
    for i, radius in enumerate(smoothing_radii):
        RL_moment[i] *= ndimage.gaussian_filter(residual**m / (compact_emission + diffuse_emission), radius)
    moment = np.zeros(n_sources+1)
    np.add.at(moment, HOT_labels, RL_moment)
    return moment

source_area = compute_moment(0)
source_residual = compute_moment(1)
mean_residual = source_residual / source_area
#rms_residual = compute_moment(2) / source_area

In [ ]:
source_SN = mean_residual * np.sqrt(source_area) / noise
sorted_by_SN = np.argsort(source_SN)
n_fluke = np.count_nonzero(np.cumsum(source_residual[sorted_by_SN]) < 0)
SN_threshold = source_SN[sorted_by_SN[n_fluke]]
true_source = (source_SN > SN_threshold) & (source_area > compact_scale)
n_true = np.count_nonzero(true_source)
print(f'{n_true} true sourcces above S/N={SN_threshold:.1f} and area={compact_scale:.1f}')

In [ ]:
'''
final_model = baseline + ndimage.gaussian_filter(mRL[1], smoothing_radii[1])

RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])

RL = np.zeros_like(data)
indices = np.where(~true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])
'''

'''
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[1]])
RL[indices] = mRL[1][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[1])
'''

final_model = baseline
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[0]])
RL[indices] = mRL[0][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[0])
RL = np.zeros_like(data)
indices = np.where(true_source[HOT_labels[1]])
RL[indices] = mRL[1][indices]
final_model += ndimage.gaussian_filter(RL, smoothing_radii[1])


In [ ]:
fig, axes = new_figure('source_flux', nrows=1, figsize=(12, 8))

ax = axes[0, 0]
ax.axhline(SN_threshold, color='k', ls=':', label=f'{SN_threshold} x {noise:.3g} = {SN_threshold*noise:.3g}')
ax.axvline(compact_scale, color='k', ls=':', label=f'compact scale = {compact_scale:.3g}')
ax.axvline(diffuse_scale, color='k', ls=':', label=f'diffuse scale = {diffuse_scale:.3g}')

ax.scatter(source_area[true_source], source_SN[true_source], c='b', s=5, alpha=1)
ax.scatter(source_area[~true_source], source_SN[~true_source], c='r', s=5, alpha=.5)
#for i in range(n_sources+1):
#    ax.text(source_residual[i], mean_residual[i], i, fontsize='x-small', clip_on=True)
ax.legend()
plt.show()

# 3. Plots

Normalisation and color maps:

In [ ]:
latin_cube = np.vstack([(np.argsort(np.random.random(n_sources))+1)/n_sources, (np.argsort(np.random.random(n_sources))+1)/n_sources, (np.argsort(np.random.random(n_sources))+1)/n_sources, np.ones(n_sources)]).T
latin_cube[0, :] = [0., 0., 0., 1.]  # background object must be black :^)
label_cmap = colors.ListedColormap(latin_cube)
label_norm = colors.Normalize(vmin=-.5, vmax=n_sources+.5)
print(f'{n_sources} unique sourcces')

In [ ]:
mRL_cmap = default_cmap
mRL_norm = colors.LogNorm(vmin=np.percentile(mRL[mRL>0], 10), vmax=np.percentile(mRL[mRL>0], 99))

In [ ]:
fig_name = 'GARLIC_test'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12, 10))
axes = fig.subplots(nrows=4, ncols=2, squeeze=False, sharex='col', gridspec_kw={'width_ratios': [1, .02], 'hspace': 0})
fig.suptitle(fig_name)
fig.set_tight_layout(True)


ax = axes[0, 0]
im = ax.imshow(mRL,
               interpolation='nearest', origin='lower', cmap=mRL_cmap, norm=mRL_norm,
              )
ax.set_aspect('auto')
ax.set_ylabel('smoothing scale')
ax.set_yticks(np.arange(smoothing_radii.size), [f'{radius:.3g}' for radius in smoothing_radii])
cb = plt.colorbar(im, cax=axes[0, 1], orientation='vertical', shrink=.9)
cb.ax.tick_params(labelsize='small')
cb.ax.set_ylabel('Richardson-Lucy intensity')


ax = axes[1, 0]
im = ax.imshow(source_SN[HOT_labels],
               interpolation='nearest', origin='lower', norm=colors.SymLogNorm(linthresh=SN_threshold), cmap='rainbow_r',
              )
ax.set_aspect('auto')
ax.set_ylabel('smoothing scale')
ax.set_yticks(np.arange(smoothing_radii.size), [f'{radius:.3g}' for radius in smoothing_radii])
cb = plt.colorbar(im, cax=axes[1, 1], orientation='vertical', shrink=.9)
cb.ax.tick_params(labelsize='small')
cb.ax.set_ylabel('S/N')


ax = axes[2, 0]
masked_labels = HOT_labels.copy()
indices = np.where(~true_source[masked_labels])
masked_labels[indices] = 0
im = ax.imshow(masked_labels,
               interpolation='nearest', origin='lower', cmap=label_cmap, norm=label_norm,
              )
ax.set_aspect('auto')
ax.set_ylabel('smoothing scale')
ax.set_yticks(np.arange(smoothing_radii.size), [f'{radius:.3g}' for radius in smoothing_radii])
cb = plt.colorbar(im, cax=axes[2, 1], orientation='vertical', shrink=.9)
cb.ax.tick_params(labelsize='small')
cb.ax.set_ylabel('source ID')


ax = axes[3, 0]
ax.set_ylabel('intensity')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
ax.grid(True, which='both')

if true_signal is not None:
    ax.plot(true_signal, 'g-', alpha=.3)
ax.plot(data, 'k-', alpha=.2)
ax.plot(baseline + diffuse_emission + compact_emission, 'b-', alpha=.5)

ax.plot(final_model, 'k-', alpha=1)
ax.plot(2*baseline + compact_emission + diffuse_emission - final_model, 'r-', alpha=1)
ax.plot(baseline + diffuse_estimate, 'g--')
ax.fill_between(np.arange(data.size), baseline + diffuse_estimate - noise, baseline + diffuse_estimate + noise, color='g', alpha=.1)



ax.set_xlabel('pixel ID')
#ax.set_xlim(5900, 6300)
#ax.set_ylim(-10, 300)
#ax.set_ylim(np.min(baseline)*.9, np.max(baseline + diffuse_estimate))

axes[-1, 1].axis('off')
plt.show()

In [ ]:
np.nansum(data), np.nansum(final_model)/np.nansum(data), np.nansum(source_residual), np.nansum(residual), np.nansum(source_residual[~true_source])

In [ ]:
class Explore_lbl_1D(object):
    
    def __init__(self, fig_name, data, estimate, label, parent):
        """Interactive display"""
        
        plt.close(fig_name)
        self.fig = plt.figure(fig_name, figsize=(12, 7))
        self.axes = self.fig.subplots(nrows=4, ncols=2, squeeze=False, sharex='col', gridspec_kw={'width_ratios': [1, .02], 'hspace': 0})
        self.fig.suptitle(fig_name)
        self.fig.set_tight_layout(True)

        self.original = data
        self.data_offset = np.nanmin(data)
        #self.RL = RL
        #self.SSF = SSF
        self.total_estimate = estimate
        self.label = label
        self.parent = parent

        self.ax_parent = self.axes[0, 0]
        self.ax_parent_cb = self.axes[0, 1]
        self.ax_lbl = self.axes[1, 0]
        self.ax_lbl_cb = self.axes[1, 1]

        self.ax_im = self.axes[2, 0]
        self.ax_cb = self.axes[2, 1]

        self.ax0 = self.axes[3, 0]
        self.ax0.plot(data, 'k-', alpha=.2)
        self.ax0.set_xlim(0, data.size)
        self.axes[3, 1].axis('off')

        self.widget = widgets.interactive(self.plot_lbl, lbl=widgets.BoundedIntText(value=0, min=0, max=n_sources, continuous_update=False))
        display(self.widget)


    def plot_lbl(self, lbl):
        #lbl = decreasing_flux[lbl_sorted]
        xlim = self.ax0.get_xlim()
        ylim = self.ax0.get_ylim()

        ax = self.ax_parent
        im = ax.imshow(self.parent[self.label],
                       interpolation='nearest', origin='lower',
                       cmap=label_cmap, norm=colors.Normalize(vmin=-.5, vmax=n_sources+.5),
                      )
        ax.set_aspect('auto')
        ax.set_yticks(np.arange(smoothing_radii.size), [f'{radius:.3g}' for radius in smoothing_radii])
        cb = plt.colorbar(im, cax=self.ax_parent_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        cb.ax.set_ylabel('parent ID')
        
        
        ax = self.ax_lbl
        im = ax.imshow(self.label,
                       interpolation='nearest', origin='lower',
                       cmap=label_cmap, norm=colors.Normalize(vmin=-.5, vmax=n_sources+.5),
                      )
        ax.set_aspect('auto')
        ax.set_yticks(np.arange(smoothing_radii.size), [f'{radius:.3g}' for radius in smoothing_radii])
        cb = plt.colorbar(im, cax=self.ax_lbl_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        cb.ax.set_ylabel('source ID')

        
        RL = np.zeros_like(mRL)
        indices = np.where(HOT_labels == lbl)
        RL[indices] = mRL[indices]

        self.ax_im.clear()
        im = self.ax_im.imshow(RL,
                               interpolation='nearest', origin='lower',
                               #cmap='gist_earth', norm=colors.LogNorm(vmin=np.min(RL[RL > 1e3*epsilon])))
                               cmap=default_cmap, norm=colors.LogNorm(vmin=np.percentile(mRL[mRL>0], 10), vmax=np.percentile(mRL[mRL>0], 99)),
                               #cmap=default_cmap, norm=colors.LogNorm(vmin=np.percentile(RL[RL>0], 1), vmax=np.percentile(RL[RL>0], 99)),
                              )
        self.ax_im.set_aspect('auto')
        cb = plt.colorbar(im, cax=self.ax_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        self.fig.canvas.draw_idle()
        

        ax = self.ax0
        ax.clear()

        ax.plot(self.original, 'k-', alpha=.3)
        ax.plot(baseline + diffuse_emission + compact_emission, 'b-', alpha=.25)        
        ax.plot(baseline + diffuse_estimate, 'g--', alpha=.25)
        ax.fill_between(np.arange(data.size), baseline + diffuse_estimate - noise, baseline + diffuse_estimate + noise, color='g', alpha=.1)

        ax.plot(np.sum(RL, axis=0) + baseline, 'r-', alpha=.25, label=f'source {lbl} ({true_source[lbl]})')
        source_fraction = get_source_model(mRL, HOT_labels, lbl) / (compact_emission + diffuse_emission)
        ax.plot(baseline + diffuse_estimate + residual * source_fraction, 'r--',
                label=f'area={source_area[lbl]:.3g} mean={mean_residual[lbl]:.3g} S/N={source_SN[lbl]:.1f}')
        ax.plot((baseline + diffuse_estimate) * source_fraction, 'g--')
        ax.plot(baseline + (compact_emission + diffuse_emission) * source_fraction, 'k-')

        ax.legend()
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')
        #ax.set_yscale('log')
        ax.set_ylim(.9*self.data_offset, 3e5)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        #self.ax0.set_xlim(5000, 5300)


if len(data.shape) == 1:
    estimate = np.empty_like(mRL)
    for i, radius in enumerate(smoothing_radii):
        estimate[i] = ndimage.gaussian_filter(mRL[i], radius)
    estimate = np.sum(estimate, axis=0)

    #for i, radius in enumerate(smoothing_radii):
    #     mRL[i] *= ndimage.gaussian_filter((boosted_data+epsilon) / (estimate+epsilon), radius)
    #RL = np.nansum(mRL, axis=0)
    #SSF_amplitude = np.nanmedian(mRL/RL[np.newaxis, :], axis=1)
    #print(rms_residual)
    x = Explore_lbl_1D(object_name, data, estimate, HOT_labels, HOT_parent)

# --- OLD STUFF ---

In [ ]:
'''
fig_name = 'bg_test'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12,4))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False)
fig.suptitle(fig_name)
fig.set_tight_layout(True)


ax = axes[0, 0]
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
ax.grid(True, which='both')

ax.plot(true_signal, 'g-', alpha=.3)
ax.plot(data, 'k-', alpha=.2)
#ax.axhline(data_offset, c='r', ls=':', alpha=.5)
ax.plot(baseline, 'y-', alpha=.5)
ax.plot(baseline + residual_above_bg, 'y:', alpha=.5)
ax.fill_between(np.arange(data.size), baseline, baseline+2*residual_above_bg, color='y', alpha=.1)


ax.set_xlabel('pixel')
#ax.set_xlim(5900, 6300)
ax.set_ylim(np.min(baseline-residual_above_bg), np.max(baseline+4*residual_above_bg)*1.1)

plt.show()
'''
,

In [ ]:
'''
estimate = baseline + np.sum(mRL, axis=0)
index_minima = scripts.diffuse_emission.find_minima(estimate)

minima = estimate[index_minima] - baseline[index_minima]
p16, p50 = np.nanpercentile(minima, [16, 50])
mu0 = p50
var0 = (p50 - p16)**2
weight = np.exp(-.5 * (minima - mu0)**2 / var0)
total_weight = np.nansum(weight)
mu1 = np.nansum(weight * minima) / total_weight
var1 = np.nansum(weight * (minima- mu1)**2) / total_weight
var = 1 / (1/var1 - 1/var0)
mu = var * (mu1/var1 - mu0/var0)
diffuse_offset = mu
print(diffuse_offset, np.sqrt(var))
'''
,

In [ ]:
'''
importlib.reload(scripts.multiscale_RL)

n_iter = 0
model = data
mean_below = 0
while n_iter < 10 and mean_residual > mean_below:
    n_iter += 1
    compact_scale, diffuse_scale, baseline, residual_above_bg = scripts.diffuse_emission.run(model, max_iter=14)
    smoothing_radii = np.array([1, compact_scale, diffuse_scale])
    print(f'--- {n_iter} --- {compact_scale}, {diffuse_scale}')
    
    mRL = scripts.multiscale_RL.run((model - baseline).clip(min=0), smoothing_radii, .01)
    compact_emission = ndimage.gaussian_filter(mRL[0], 1) + ndimage.gaussian_filter(mRL[1], compact_scale)
    diffuse_residual = ndimage.gaussian_filter(mRL[2], diffuse_scale)
    diffuse_emission = baseline + diffuse_residual
    model = compact_emission + diffuse_emission
    total = np.nansum(data - baseline)
    mean_residual = np.nanmean(diffuse_residual)
    mean_below = np.sqrt(np.nanmean(((baseline - data)**2)[data < baseline]))
    print(np.nansum(compact_emission)/total, mean_residual, mean_below)
'''
,

## Individual sources

RL_flux = np.zeros(n_sources+1)
RL_compact_flux = np.zeros(n_sources+1)
RL_diffuse_flux = np.zeros(n_sources+1)
RL_variance = np.zeros(n_sources+1)
np.add.at(RL_flux, HOT_labels, mRL)
np.add.at(RL_compact_flux, HOT_labels[0], np.max(mRL, axis=0))
#np.add.at(RL_diffuse_flux, HOT_labels[1], mRL[1])
np.add.at(RL_diffuse_flux, HOT_labels[0], np.min(mRL, axis=0))  # dirty hack?
np.add.at(RL_variance, HOT_labels[0], noise**2)


sorted_by_flux = np.argsort(RL_compact_flux)
RL_background = np.sum(mRL, axis=0)
RL_sources = np.zeros_like(data)
model = ndimage.gaussian_filter(RL_background, diffuse_scale)
#chi2 = np.sqrt(np.nanmean(((data - background - model) / noise)**2))
chi2 = np.nanmean(((data - background - model) / noise)**2)
print(chi2, 1+1/np.sqrt(data.size))
for lbl in sorted_by_flux[::-1]:
    old_chi2 = chi2
    indices = np.where(HOT_labels[0] == lbl)
    RL_this_source = np.fmax(mRL[0][indices], mRL[1][indices])
    RL_sources[indices] += RL_this_source
    RL_background[indices] -= RL_this_source
    #RL_sources[indices] += mRL[0][indices]
    #RL_background[indices] -= mRL[0][indices]
    #indices = np.where(HOT_labels[1] == lbl)
    #RL_sources[indices] += mRL[1][indices]
    #RL_background[indices] -= mRL[1][indices]
    model = ndimage.gaussian_filter(RL_background, smoothing_radii[1])
    model += ndimage.gaussian_filter(RL_sources, smoothing_radii[0])
    chi2 = np.sqrt(np.nanmean(((data - background - model) / noise)**2))
    print(lbl, RL_compact_flux[lbl], chi2)
    #if chi2 > old_chi2:
    #    break

sorted_by_flux = np.argsort(RL_compact_flux)
true_source = np.zeros_like(HOT_parent, dtype=bool)
true_source[sorted_by_flux] = np.cumsum(RL_compact_flux[sorted_by_flux]) > RL_flux_threshold

#RL_flux_threshold = np.sum((mRL[1] - mRL[0])[mRL[0] < mRL[1]])
#sorted_by_flux = np.argsort(RL_compact_flux - RL_diffuse_flux)
RL_flux_threshold = np.sum(compact_emission + diffuse_emission - diffuse_estimate)
sorted_by_flux = np.argsort(RL_flux)

true_source = np.zeros_like(HOT_parent, dtype=bool)
true_source[sorted_by_flux[::-1]] = np.cumsum(RL_flux[sorted_by_flux[::-1]]) <= RL_flux_threshold

np.sum(mRL[1]), true_source

RL_background = np.zeros_like(data)
RL_background[~true_source[HOT_labels[0]]] = mRL[0][~true_source[HOT_labels[0]]]
#RL_background[~true_source[HOT_labels[1]]] += mRL[1][~true_source[HOT_labels[1]]]

RL_sources = np.zeros_like(data)
RL_sources[true_source[HOT_labels[0]]] = mRL[0][true_source[HOT_labels[0]]]
RL_sources[true_source[HOT_labels[1]]] += mRL[1][true_source[HOT_labels[1]]]


np.sum(mRL), np.sum(RL_sources), np.sum(RL_background), np.sum(RL_sources + RL_background)

RL_compact_flux[true_source]

fig, axes = new_figure('source_flux', nrows=3, figsize=(12, 8))

ax = axes[0, 0]
ax.plot(RL_compact_flux[sorted_by_flux], np.sqrt(2*np.cumsum(RL_variance[sorted_by_flux])), 'r--')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_compact_flux[sorted_by_flux]), 'b-')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_diffuse_flux[sorted_by_flux]), 'r-')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum((RL_compact_flux - RL_diffuse_flux)[sorted_by_flux]), 'k-')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum((RL_compact_flux - RL_diffuse_flux)[sorted_by_flux]) - np.sqrt(2*np.cumsum(RL_variance[sorted_by_flux])), 'k:')
ax.axhline(np.sum(mRL[1]), c='b', ls=':')
ax.axhline(RL_flux_threshold, c='k', ls='--')
#ax.axhline(np.sum(mRL[0]-mRL[1]), c='orange')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_diffuse_flux[sorted_by_flux]) + np.sqrt(np.cumsum(RL_variance[sorted_by_flux])), c='r', ls=':')
ax.plot(RL_compact_flux[sorted_by_flux], np.cumsum(RL_compact_flux[sorted_by_flux]) - np.sqrt(np.cumsum(RL_variance[sorted_by_flux])), c='b', ls=':')

ax = axes[1, 0]
ax.plot(RL_compact_flux[~true_source], RL_diffuse_flux[~true_source], 'ro')
ax.plot(RL_compact_flux[true_source], RL_diffuse_flux[true_source], 'co')
for i in range(n_sources+1):
    ax.text(RL_compact_flux[i], RL_diffuse_flux[i], i, fontsize='x-small', clip_on=True)
ax.plot([0, np.max(RL_compact_flux)], [0, np.max(RL_compact_flux)], 'k--')

ax = axes[2, 0]
ax.plot(RL_compact_flux[~true_source], RL_compact_flux[~true_source] / np.sqrt(RL_variance[~true_source]), 'ro')
ax.plot(RL_compact_flux[true_source], RL_compact_flux[true_source] / np.sqrt(RL_variance[true_source]), 'co')
for i in range(n_sources+1):
    ax.text(RL_compact_flux[i], RL_compact_flux[i] / np.sqrt(RL_variance[i]), i, fontsize='x-small', clip_on=True)


#raise -1

def get_mRL_inpaint(mRL, labels, target):
    raise -1


def get_mRL_inpaint_OLD(mRL, labels, target):

    mRL_target = np.zeros_like(mRL)
    indices = np.where(labels == target)
    mRL_target[indices] = mRL[indices]
    #print(f'mRL {target} (area = {len(indices[0])}): {np.nanmax(mRL_target)} {np.nanmax(mRL[indices])}')

    radius = np.sqrt(mRL.shape[0])
    inpaint_map = np.where(mRL_target > 0, 1., 0.)  # object mask
    if len(inpaint_map[0]) > 0:
        inpaint_map = ndimage.gaussian_filter(inpaint_map, radius)  # interpolation weight
        inpaint_map = np.where(inpaint_map > 0, inpaint_map, np.nan)  # to prevent division by zero
        inpaint_map = ndimage.gaussian_filter(mRL_target, radius) / inpaint_map
    else:
        inpaint_map = mRL_target

    return mRL_target, np.fmin(inpaint_map, mRL)


def get_individual_mRL_OLD(mRL, labels, parent, target):

    mRL_target, inpaint_map = get_mRL_inpaint(mRL, labels, target)
    #print(f'mRL {target}: {np.nanmax(mRL_target)}')
    
    # compute contribution to descendants:
    progenitor = np.array(parent[labels])
    if parent[target] == target:
        indices = np.where(labels == target)
        progenitor[indices] = 0
    n_found = 1
    while n_found > 0:
        indices = np.where(progenitor == target)
        n_found = len(indices[0])
        #print(f'{n_found} values painted for {target} ({np.unique(parent[progenitor][indices])})')
        #print(f'{np.unique(labels[indices])}')
        #print(f'{parent[np.unique(labels[indices])]}')
        mRL_target[indices] = inpaint_map[indices]
        progenitor[indices] = 0
        progenitor = parent[progenitor]

    # remove contribution from ancestors:
    inpaint_map = get_mRL_inpaint(mRL, labels, parent[target])[1]
    indices = np.where(labels == target)
    #print(f'Removing {parent[target]} from {target}: {np.nanmax(mRL_target[indices])} {np.nanmax(inpaint_map[indices])}')
    mRL_target[indices] -= inpaint_map[indices]

    return mRL_target


def get_individual_mRL_another(mRL, labels, parent, target):

    mRL_target = np.zeros_like(mRL)
    indices = np.where(labels == target)
    mRL_target[indices] = mRL[indices] - HOT_bg[target]
    # add descendants background:
    progenitor = np.array(parent[labels])
    if parent[target] == target:
        indices = np.where(labels == target)
        progenitor[indices] = 0
        mRL_target[indices] += HOT_bg[target]
    n_found = 1
    while n_found > 0:
        indices = np.where(progenitor == target)
        n_found = len(indices[0])
        #print(f'{n_found} values painted for {target} ({np.unique(parent[progenitor][indices])})')
        #print(f'{np.unique(labels[indices])}')
        #print(f'{parent[np.unique(labels[indices])]}')
        mRL_target[indices] = HOT_bg[labels][indices]
        progenitor[indices] = 0
        progenitor = parent[progenitor]
        n_found = 0  # only one generation
    '''
    '''

def get_individual_mRL(mRL, labels, parent, target):

    mRL_target = np.zeros_like(mRL)
    indices = np.where(labels == target)
    mRL_target[indices] = mRL[indices]

    return mRL_target